In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 HParams 仪表板的 Vertex AI TensorBoard 超参数调整

<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_hyperparameter_tuning_with_hparams.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> <br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftensorboard%2Ftensorboard_hyperparameter_tuning_with_hparams.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"> <br> 在 Colab Enterprise 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_hyperparameter_tuning_with_hparams.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> <br>
      在 GitHub 上查看
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tensorboard/tensorboard_hyperparameter_tuning_with_hparams.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> <br>
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

注意：已在以下环境中测试了此笔记本：

Python 版本 = 3.8

## 概述

### 什么是Vertex AI TensorBoard

Vertex AI TensorBoard是[开源TensorBoard](https://www.tensorflow.org/tensorboard/get_started)（TB）的企业级托管版本，TensorBoard是谷歌开源的用于机器学习实验可视化的项目。

Vertex AI TensorBoard提供各种详细的可视化，包括以下内容：

- 跟踪和可视化指标，如损失和准确性随时间的变化，
- 可视化模型计算图（操作和层），
- 查看随时间变化的权重、偏差或其他张量的直方图，
- 将嵌入投影到低维空间，
- 显示图像、文本和音频样本。

除了来自TensorBoard强大的可视化之外，Vertex AI TensorBoard还提供以下好处：

- 实验仪表板的持久性、可共享链接，
- 项目中所有实验的可搜索列表，
- 与Vertex AI服务的模型训练集成，
- 企业级安全性、隐私和合规性。

通过Vertex AI TensorBoard，您可以跟踪、可视化和比较机器学习实验，并与团队共享。

了解更多关于[Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction)。

### 目标

这个教程演示了如何在TensorFlow中记录超参数实验结果，并在TensorBoard的Hparams仪表板中可视化结果。

这个教程使用了以下Vertex AI服务和资源：

- Vertex AI TensorBoard

执行的步骤包括：

* 调整TensorFlow运行以记录超参数和指标。
* 启动运行并将它们全部记录在一个父目录下。
* 在TensorBoard的HParams仪表板中可视化结果。

数据集

本教程使用[FashionMNIST](https://github.com/zalandoresearch/fashion-mnist)数据集。

费用

本教程使用谷歌云的以下可计费组件：

* Vertex AI

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing),
并使用[定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

安装依赖项

安装下面这些包，这些包是运行本教程笔记本所需的。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[tensorboard] \
                                 tensorflow

### 仅限 Colab：取消下面单元格的注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的 Google Cloud 项目

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得一个可用于抵消计算/存储成本的 300 美元免费信用额度。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，请安装 [Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 设置区域

**可选**: 更新‘REGION’变量以指定您想要使用的区域。了解更多关于[Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的谷歌云账号

要认证您的谷歌云账号，请按照您Jupyter环境中的指示操作。

**Vertex AI 工作台**
您已通过验证。

本地的JupyterLab实例
取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

Translate the following English text to Chinese: * **Colab**
取消注释并运行以下代码：

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

### 导入库

In [ ]:
from google.cloud import aiplatform

### 初始化 Python 的 Vertex AI SDK

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

加载TensorBoard和TensorFlow组件

加载TensorBoard笔记本扩展并导入TensorFlow和TensorBoard HParams插件。

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/

# Import TensorFlow and the TensorBoard HParams plugin
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

下载数据集

下载[FashionMNIST](https://github.com/zalandoresearch/fashion-mnist)数据集并对其进行缩放。

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 设置实验

通过指定以下超参数的值来运行实验：

* 第一个密集层中的单位数，
* dropout层中的dropout率，
* 优化器。

在TensorBoard中指定实验的超参数值。

* 可选：为了更细粒度地过滤Google Cloud控制台中的超参数，提供域信息并指定应显示哪些指标。

In [ ]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "sgd"]))

METRIC_ACCURACY = "accuracy"

with tf.summary.create_file_writer("logs/hparam_tuning").as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy")],
    )

将TensorFlow运行适应以记录超参数和指标

模型将非常简单：两个密集层之间带有一个dropout层。训练代码将看起来很熟悉，尽管超参数不再硬编码。相反，超参数以`hparams`字典的形式提供，并在整个训练函数中使用：

In [ ]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax),
        ]
    )
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    model.fit(
        x_train, y_train, epochs=1
    )  # Run with 1 epoch to speed things up for demo purposes
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

对于每次运行，记录一个包含超参数和最终准确度的hparams摘要。

In [ ]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

## 开始运行并将它们全部记录在一个父目录下

你现在可以尝试多个实验，每个实验使用不同的超参数集进行训练。

为简单起见，可以使用网格搜索：尝试所有离散参数的组合，以及实值参数的下限和上限。对于更复杂的情况，可能更有效的是随机选择每个超参数值（这被称为随机搜索）。还有更高级的方法可供选择。

运行几个实验，这将需要几分钟。

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print("--- Starting trial: %s" % run_name)
            print({h.name: hparams[h] for h in hparams})
            run("logs/hparam_tuning/" + run_name, hparams)
            session_num += 1

在 Vertex AI TensorBoard 的 HParams 选项卡中可视化结果。

创建 Vertex AI TensorBoard
必须先创建一个区域化资源，存储您的 Vertex AI TensorBoard 实验的 Vertex AI TensorBoard 实例，然后才能对实验进行可视化。您可以在一个项目中创建多个实例。

详细了解请参见[创建 Vertex AI TensorBoard 实例](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-setup#create-tensorboard-instance)。

创建一个用于训练作业的 TensorBoard 实例。

In [ ]:
TENSORBOARD_NAME = "[your-tensorboard-name]"  # @param {type:"string"}

if (
    TENSORBOARD_NAME == ""
    or TENSORBOARD_NAME is None
    or TENSORBOARD_NAME == "[your-tensorboard-name]"
):
    TENSORBOARD_NAME = PROJECT_ID + "-tb-"

tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=REGION
)
TENSORBOARD_RESOURCE_NAME = tensorboard.gca_resource.name
print("TensorBoard resource name:", TENSORBOARD_RESOURCE_NAME)

设置您的TensorBoard实验名称。

In [ ]:
from datetime import datetime

EXPERIMENT_NAME = "[your-experiment-run-name]"  # @param {type:"string"}

if (
    EXPERIMENT_NAME == ""
    or EXPERIMENT_NAME is None
    or EXPERIMENT_NAME == "[your-experiment-run-name]"
):
    EXPERIMENT_NAME = "experiment" + datetime.now().strftime("%H-%M-%S")

将日志上传至您的Vertex AI TensorBoard

In [ ]:
!tb-gcp-uploader --one_shot=True --tensorboard_resource_name=$TENSORBOARD_RESOURCE_NAME --logdir="logs/hparam_tuning/" --experiment_name=$EXPERIMENT_NAME

点击生成的TensorBoard链接，并在顶部点击“HParams”。

仪表板的左侧窗格提供了在HParams仪表板的所有视图中均有效的过滤功能：

- 过滤在仪表板中显示哪些超参数/指标
- 过滤在仪表板中显示哪些超参数/指标值
- 根据运行状态（运行中，成功，...）进行过滤
- 在表格视图中按超参数/指标进行排序
- 显示要显示的会话组数（在有许多实验时提高性能时有用）

HParams仪表板有三种不同的视图，提供各种有用的信息：

* **表格视图**列出运行、它们的超参数和指标。
* **平行坐标视图**将每个运行显示为通过每个超参数和指标轴的线。单击并拖动鼠标到任何轴上以标记一个区域，这将仅突出显示通过该区域的运行。这对于识别哪些超参数组最重要可能很有用。轴本身可以通过拖动来重新排序。
* **散点图视图**显示了比较每个超参数/指标与每个指标的图。这可以帮助识别相关性。单击并拖动以选择特定图中的区域，并突出显示其他图中的那些会话。

可以单击表行、平行坐标线和散点图标记，以查看指标随训练步骤变化的图（尽管在本教程中每次运行仅使用一个步骤）。

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
import os

# Delete endpoint resource
# e.g. `endpoint.delete()`

# Delete model resource
# e.g. `model.delete()`

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI